In [ ]:
import pandas as pd
df = pd.read_csv('testing_data.csv')
df

In [ ]:
df = df.fillna(method='ffill').set_index('date')


In [ ]:
# create rsv
rsv = (df - df.rolling(window=9).min())/(df.rolling(window=9).max()- df.rolling(window=9).min())*100

In [ ]:
rsv = rsv.fillna(0)


In [ ]:
# create K dataframe
# K = 2/3 X (昨日K值) + 1/3 X (今日RSV)
k_list=[]
k = 50

for i in range(0,len(rsv.index)):
    if i<=7:
        k_list.append(np.full(shape=10,fill_value=50,dtype=np.int))
    else:
        k = (2/3)*k + (1/3)*rsv.values[i,:]
        k_list.append(k)

k_df = pd.DataFrame(k_list, columns=df.columns, index=df.index)
k_df

In [ ]:
# create D dataframe
# D = 2/3 X (昨日D值) + 1/3 X (今日K值)
d_list=[]
d = 50

for i in range(0,len(k_df.index)):
    if i<=7:
        d_list.append(np.full(shape=10,fill_value=50,dtype=np.int))
    else:
        d = (2/3)*d + (1/3)*k_df.values[i,:]
        d_list.append(d)

d_df = pd.DataFrame(d_list, columns=df.columns, index=df.index)
d_df

In [ ]:
#rsi
temp = df-df.shift(1)
temp

In [ ]:
rsi_pos = temp[temp>0].fillna(0)
rsi_neg = -temp[temp<0].fillna(0)

# create RSI(相對強弱指標):以12天為基準
# RSI = (前N日漲幅平均值/(前N日漲幅平均值+前N日跌幅平均值))×100
up_mean_12 = []
down_mean_12 = []
for i in range(13,len(df)+1):
    up_mean_12.append(rsi_pos.values[i-12:i].mean(axis=0))
    down_mean_12.append(rsi_neg.values[i-12:i].mean(axis=0))
    
rsi_list = []
for i in range(len(up_mean_12)):
    rsi_list.append(100*up_mean_12[i] / (up_mean_12[i]+down_mean_12[i]))

rsi_df = pd.DataFrame(rsi_list, columns=df.columns)
rsi_df

In [ ]:
a = np.empty((12,10,))
a[:] = np.nan
a = pd.DataFrame(a,columns=df.columns)

rsi_combine = pd.concat([a,rsi_df])
rsi_combine.index =[df.index]
rsi_combine = rsi_combine.fillna(0)
rsi_combine

In [ ]:
k_df.to_csv('test_k_value.csv')

In [ ]:
d_df.to_csv('test_d_value.csv')

In [ ]:
rsi_combine.to_csv('test_rsi_value.csv')